In [1]:
from config import parameters
from utils import *

base_dir: /home/dmlab/jihye/GIT/bis_speeches_text_bigram_frequency
Parameters(
  sentence_length_outlier = [0, 1, 2, 181, 252]
  base_dir = /home/dmlab/jihye/GIT/bis_speeches_text_bigram_frequency
  data_dir = /home/dmlab/jihye/GIT/bis_speeches_text_bigram_frequency/data
  output_base_dir = /home/dmlab/jihye/GIT/bis_speeches_text_bigram_frequency/output
  bis_raw_pkl_filepath = /home/dmlab/jihye/GIT/bis_speeches_text_bigram_frequency/data/bis_w_content_FINAL.pkl
)


### User configuration

In [10]:
bigram_window_size = 15
bigram_max_rank = None

stopword_list = ['financial', 'market', 'federal', 'bank', 'banking', 'bi', 'review', 'year', 'reserve', 'policy', 'state', 'central', 'board', 'percent'
               , 'mr', 'alan', 'greenspan', 'ben', 'bernanke', 'janet', 'yellen', 'jerome', 'powell', 'vol'
              , 'ha', 'wa']  
ws_quarterly, ws_semiannually, ws_annually = 3, 1, 0

In [8]:
stopset, filter_stops = stopwords_set_filter('english', stopword_list)

quarterly_filepath = 'target-list-units-grouped_quarterly_20191028-21-52-08.pkl'
semiannually_filepath = 'target-list-units-grouped_semiannually_20191028-21-52-09.pkl'
annually_filepath = 'target-list-units-grouped_annually_20191028-21-52-09.pkl'
quarterly_target_doc_dict = load_pkl(os.path.join(parameters.output_base_dir, quarterly_filepath))
semiannually_target_doc_dict = load_pkl(os.path.join(parameters.output_base_dir, semiannually_filepath))
annually_target_doc_dict = load_pkl(os.path.join(parameters.output_base_dir, annually_filepath))

period_dict = {'quarterly': (quarterly_target_doc_dict, ws_quarterly), 'semiannually': (semiannually_target_doc_dict, ws_semiannually), 'annually': (annually_target_doc_dict, ws_annually)}

Completed loading: /home/dmlab/jihye/GIT/bis_speeches_text_bigram_frequency/output/target-list-units-grouped_quarterly_20191028-21-52-08.pkl
Completed loading: /home/dmlab/jihye/GIT/bis_speeches_text_bigram_frequency/output/target-list-units-grouped_semiannually_20191028-21-52-09.pkl
Completed loading: /home/dmlab/jihye/GIT/bis_speeches_text_bigram_frequency/output/target-list-units-grouped_annually_20191028-21-52-09.pkl


In [68]:
from BigramCollocationFinder_custom import BigramCollocationFinder

def words_from_pos_tagged_words(pos_tagged_words):
    return [x.strip() for (x,y) in pos_tagged_words]

def get_one_dimensional_words(doc_list):
    words = list()
    for doc in doc_list:
        for unigrams_with_pos in doc['unigrams_by_sentence']:
            words.extend(words_from_pos_tagged_words(unigrams_with_pos))
    return words

def bigram_freq_rank_dict(finder, bigram_max_rank=None):
    bigram_dict = dict()
    _rank = 0
    for _bigram, _freq in sorted(finder.ngram_fd.items(), key=lambda t:t[-1], reverse=True):   # frequency descending
        _rank +=1 
        if _bigram[0] == _bigram[1]:
            _rank -=1
            continue
        bigram_dict[_bigram[0]+'-'+_bigram[1]] = (_freq, _rank)
        if bigram_max_rank is not None and _rank == bigram_max_rank:
            break
    return bigram_dict

def nltk_bigram_collocation_finder(word_list, bigram_window_size=2, filter_stops=None):        
    finder = BigramCollocationFinder.from_words(word_list, window_size=bigram_window_size) 
    if filter_stops is not None:
        finder.apply_word_filter(filter_stops) 
    return finder

In [69]:
final_bigram_freq_rank_dict = dict()
for _period_category, (_period_dict, _ws) in period_dict.items():
    print('='*5, '\n', _period_category)
    for i, _period in enumerate(sorted(_period_dict.keys())):
        if i-_ws < 0:
            continue
            
        # Bigram frequency
        _current_periods = [j for j in range(i-_ws, i+1)]
        _current_period_list_of_list_of_doc = [_period_dict[_period_j] for j, _period_j in enumerate(sorted(_period_dict.keys())) if j in _current_periods]
        _current_doc_list = list()
        for _current_period_list_of_doc in _current_period_list_of_list_of_doc:
            for doc in _current_period_list_of_doc:
                _current_doc_list.append(doc)
        _current_words = get_one_dimensional_words(_current_doc_list)
            
        _current_finder = nltk_bigram_collocation_finder(_current_words, bigram_window_size, filter_stops)
        _current_bigram_freq_rank_dict = bigram_freq_rank_dict(_current_finder, bigram_max_rank)
        
        final_bigram_freq_rank_dict[_period_category] = dict()
        final_bigram_freq_rank_dict[_period_category][_period] = _current_bigram_freq_rank_dict
        # BIGRAM FREQUENCY 완료

===== 
 quarterly
===== 
 semiannually
===== 
 annually


In [74]:
for _bigram in list(_current_bigram_freq_rank_dict.keys())[:10]:
    print(_bigram, '\t', _current_bigram_freq_rank_dict[_bigram])

debt-business 	 (59, 1)
sheet-balance 	 (53, 2)
rate-interest 	 (52, 3)
labor-force 	 (48, 4)
participation-labor 	 (45, 5)
productivity-growth 	 (39, 6)
rural-community 	 (36, 7)
development-community 	 (35, 8)
participation-force 	 (35, 9)
rate-elb 	 (32, 10)
